In [52]:
import pickle
import os, sys
import subprocess
import pandas as pd
import streamlit as st
os.chdir('/home/p.zanineli/work/inference')

In [2]:
from modules.eval import load,forces_load,eval_report

In [3]:
def load_module(path,prop):
    dft,ml = load(f'{path}/dft_{prop}.pkl'),load(f'{path}/predicted_{prop}.pkl')
    return [ml,dft]

In [78]:
path = 'experiments/zirconium-dioxide/results'
mlips = os.listdir(path)

if './ipycheckpoints' in mlips: mlips.remove('./ipycheckpoints')
if './metrics.csv' in mlips: mlips.remove('./metrics.csv')

In [65]:
model_count = []
dict_data = {}

for mlip in mlips:
    models = os.listdir(f'{path}/{mlip}')
    
    for model in models:

        if model == 'GRACE-1L-OAM': break
        
        model_path = f'{path}/{mlip}/{model}'
        
        energies = load_module(model_path,'energies')
        forces = load_module(model_path,'forces')

        force_modules = []
        for force in forces:
            force_module,x,y,z = forces_load(force)
            force_modules.append(force_module)
        
        e_r2,e_mae,e_rmse = eval_report(energies[0],energies[1])
        f_r2,f_mae,f_rmse = eval_report(force_modules[0],force_modules[1])

        dict_data[model] = [e_r2,e_mae,e_rmse,f_r2,f_mae,f_rmse]

        model_count.append(mlip)

In [68]:
df = pd.DataFrame(dict_data).T
df.columns = ['e_r2','e_mae','e_rmse','f_r2','f_mae','f_rmse']

df = df.reset_index().rename(columns={"index": "Name"})

df.insert(1, "Label", model_count)

multi_columns = [
    ("Model", "Name"), ("Model", "Label"),
    ("Energy", "e_r2"), ("Energy", "e_mae"), ("Energy", "e_rmse"),
    ("Forces", "f_r2"), ("Forces", "f_mae"), ("Forces", "f_rmse")
]

df.columns = pd.MultiIndex.from_tuples(multi_columns)

In [77]:
df.to_csv(f'{path}/metrics.csv',index=False)

In [86]:
subprocess.run(
    "streamlit run experiments/run-dashboard.py --server.address=localhost --server.port=8501",
    shell=True
)




  You can now view your Streamlit app in your browser.

  URL: http://localhost:8501

  Stopping...


KeyboardInterrupt: 